In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'brain-tumor-mri-dataset' dataset.
Path to dataset files: /kaggle/input/brain-tumor-mri-dataset


In [ ]:
import os
import numpy as np
from PIL import Image

def carregar_dataset(base_dir, size=(64, 64)):
    X, y = [], []
    labels = sorted(os.listdir(base_dir))
    label_map = {label: i for i, label in enumerate(labels)}

    for label in labels:
        folder = os.path.join(base_dir, label)
        for file in os.listdir(folder):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img = Image.open(os.path.join(folder, file)).convert('L')  # escala de cinza
                img = img.resize(size)
                X.append(np.array(img) / 255.0)  # normaliza 0-1
                y.append(label_map[label])
    return np.array(X), np.array(y), label_map

X, y, labels = carregar_dataset(f"{path}/Training")
print("Shape X:", X.shape)
print("Classes:", labels)


Shape X: (5712, 64, 64)
Classes: {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


In [ ]:
# ========================
#  CAMADAS MANUAIS
# ========================
def conv2d(image, kernel):
    k = kernel.shape[0]
    out = np.zeros((image.shape[0] - k + 1, image.shape[1] - k + 1))
    for i in range(out.shape[0]):
        for j in range(out.shape[1]):
            out[i, j] = np.sum(image[i:i+k, j:j+k] * kernel)
    return out

def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def max_pooling(image, size=2):
    out = np.zeros((image.shape[0]//size, image.shape[1]//size))
    mask = np.zeros_like(image)

    for i in range(0, image.shape[0], size):
        for j in range(0, image.shape[1], size):
            block = image[i:i+size, j:j+size]
            m = np.max(block)
            out[i//size, j//size] = m

            # salva posição do máximo p/ backprop
            mi, mj = np.where(block == m)
            mask[i + mi[0], j + mj[0]] = 1

    return out, mask

def max_pooling_backprop(d_out, mask, size=2):
    d_img = np.zeros_like(mask)
    for i in range(d_out.shape[0]):
        for j in range(d_out.shape[1]):
            # espalha gradiente só na posição do máximo
            block = mask[i*size:(i+1)*size, j*size:(j+1)*size]
            pos = np.where(block == 1)
            d_img[i*size + pos[0][0], j*size + pos[1][0]] = d_out[i, j]
    return d_img

def softmax(x):
    e = np.exp(x - np.max(x))
    return e / np.sum(e)


In [ ]:
np.random.seed(42)

kernel = np.random.randn(3, 3) * 0.1

sample = X[0]
conv_out = conv2d(sample, kernel)
relu_out = relu(conv_out)
pool_out, _ = max_pooling(relu_out)
flat_size = pool_out.size

W = np.random.randn(flat_size, 4) * 0.1
b = np.zeros(4)

lr = 0.001
epochs = 5


In [ ]:
for epoch in range(epochs):
    loss_total = 0

    for n in range(len(X)):
        img = X[n]
        y_true = np.zeros(4)
        y_true[y[n]] = 1

        # ----------------------
        #   FORWARD PASS
        # ----------------------
        conv_out = conv2d(img, kernel)
        relu_out = relu(conv_out)
        pool_out, pool_mask = max_pooling(relu_out)
        flat = pool_out.flatten()

        logits = np.dot(flat, W) + b
        probs = softmax(logits)

        # perda
        loss = -np.sum(y_true * np.log(probs + 1e-8))
        loss_total += loss

        # ----------------------
        #   BACKPROP
        # ----------------------

        # saída → logits
        d_logits = probs - y_true

        # dW e db da camada densa
        dW = np.outer(flat, d_logits)
        db = d_logits

        # backprop p/ pooling flatten
        d_flat = np.dot(W, d_logits)
        d_pool = d_flat.reshape(pool_out.shape)

        # backprop do maxpooling
        d_relu = max_pooling_backprop(d_pool, pool_mask)

        # backprop ReLU
        d_conv = d_relu * relu_deriv(conv_out)

        # backprop convolução → kernel
        d_kernel = np.zeros_like(kernel)
        k = kernel.shape[0]

        for i in range(d_conv.shape[0]):
            for j in range(d_conv.shape[1]):
                region = img[i:i+k, j:j+k]
                d_kernel += region * d_conv[i, j]

        # ----------------------
        #   Atualização dos pesos
        # ----------------------
        W -= lr * dW
        b -= lr * db
        kernel -= lr * d_kernel

    print(f"Época {epoch+1}, perda = {loss_total/len(X):.4f}")


Época 1, perda = 0.1466
Época 2, perda = 0.1783
Época 3, perda = 0.1253
Época 4, perda = 0.0983
Época 5, perda = 0.0849


In [ ]:
def predict(image):
    c = conv2d(image, kernel)
    r = relu(c)
    p, _ = max_pooling(r)
    flat = p.flatten()
    probs = softmax(np.dot(flat, W) + b)
    return np.argmax(probs), probs


In [ ]:
X_test, y_test, _ = carregar_dataset(f"{path}/Testing")
print("Shape X_test:", X_test.shape)


Shape X_test: (1311, 64, 64)


In [ ]:
inv_labels = {v: k for k, v in labels.items()}  # para ler nomes de classes

for i in range(50):  # mostra as 5 primeiras imagens do teste
    pred, probs = predict(X_test[i])

    print("Classe verdadeira:", inv_labels[y[i]])
    print("Classe predita:", inv_labels[pred])
    print("Probabilidades por classe:")
    for i, c in inv_labels.items():
        print(f"{c:10s}: {probs[i]:.3f}")
    print("\n\n")



Classe verdadeira: glioma
Classe predita: pituitary
Probabilidades por classe:
glioma    : 0.008
meningioma: 0.025
notumor   : 0.010
pituitary : 0.957



Classe verdadeira: glioma
Classe predita: pituitary
Probabilidades por classe:
glioma    : 0.073
meningioma: 0.268
notumor   : 0.307
pituitary : 0.351



Classe verdadeira: glioma
Classe predita: pituitary
Probabilidades por classe:
glioma    : 0.001
meningioma: 0.002
notumor   : 0.003
pituitary : 0.993



Classe verdadeira: glioma
Classe predita: pituitary
Probabilidades por classe:
glioma    : 0.005
meningioma: 0.014
notumor   : 0.008
pituitary : 0.973



Classe verdadeira: glioma
Classe predita: pituitary
Probabilidades por classe:
glioma    : 0.105
meningioma: 0.110
notumor   : 0.292
pituitary : 0.493



Classe verdadeira: glioma
Classe predita: pituitary
Probabilidades por classe:
glioma    : 0.000
meningioma: 0.000
notumor   : 0.001
pituitary : 0.999



Classe verdadeira: glioma
Classe predita: pituitary
Probabilidades por class